##### Imports 

In [1]:
import pandas as pd
import csv
import logging
#from functions_py.mephys_funcs import read_file, merge_dataframes, filter_date, drop_cols, drop_nans, \
#create_cond_df, create_container_df, filter_df

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(context = "notebook", style = "ticks", font="verdana") # font_scale = 1.35)

##### Logging Set-up

In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

##### Pandas Display Settings

In [3]:
pd.set_option("display.max_colwidth",150) #Expands the number of characters shown in the columns
pd.set_option('display.max_columns', None)

##### Lists/Paths

In [4]:
fields_jem = ["date", "organism_name", "container", "rigOperator", "status", "roi",
              "extraction.postPatch", "extraction.endPipetteR"]
fields_ephys = ["name", "vrest", "ri", "sag", 
                "tau", "upstroke_downstroke_ratio_long_square", "latency", "f_i_curve_slope"]
fields_shiny = ["cell_name", "sample_id", "cell_specimen_project",
                "subclass_label", "topLeaf_label", "broad_class_label", "VISp_cluster",
                "marker_sum_norm_label", "Norm_Marker_Sum.0.4_label", "Tree_call"]

In [5]:
#path_jem = "Z:/Patch-Seq/compiled-jem-data/jem_metadata.csv"
#path_shiny = "//allen/programs/celltypes/workgroups/rnaseqanalysis/shiny/patch_seq/star/mouse_patchseq_VISp_current/mapping.df.with.bp.40.lastmap.csv"

path_jem = "C:/Users/kumar/Documents/GitHub/analysis_projects/csv/jem_metadata_wFAILURE.csv"
path_ephys = "C:/Users/kumar/Documents/GitHub/analysis_projects/csv/ephys_mIVSCC_MET.csv"
path_shiny = "C:/Users/kumar/Documents/GitHub/analysis_projects/csv/Mouse_VISp_ctx_shiny.csv"

In [6]:
drop_cols_list = ["name", "patched_cell_container", "cell_name_label", "sample_id"]
drop_nans_list = ["date", "name", "rigOperator"]

In [7]:
r_users = ["kristenh", "lindsayn", "ramr", "katherineb", "jessicat"]

In [8]:
b_colors={"RSP": "#a1d99b", "VISp": "#9ecae1"}
s_colors={"RSP": "#41ab5d", "VISp": "#4292c6"}

##### Functions

In [9]:
def read_file(path, fields=None):
    """Reads file in as pandas dataframe by using pd.read_csv
    Args:
        path: path of file location
    Return:
        df: a pandas dataframe
    """
    global df
    df = pd.read_csv(path, usecols=fields)
    LOGGER.info("Read file in as a pandas dataframe")
    return df

In [10]:
def merge_dataframes(left_df, right_df, left_col, right_col, join_how):
    """Merges two dataframes together into one dataframe
    Args:
        left_df: a pandas dataframe on the left
        right_df: a pandas dataframe on the right
        left_col: a column from the left dataframe
        right_col: a column from the right dataframe
    Return:
        merge_df: a merged pandas dataframe
    """
    merge_df = pd.merge(left = left_df,
                        right = right_df,
                        left_on = left_col,
                        right_on = right_col, 
                        how = join_how)
    LOGGER.info("Merged two pandas dataframe into one dataframe")
    return merge_df

In [11]:
def drop_cols(df, drop_col):
    """Drop unnessary columns from dataframe
    Args:
        df: a pandas dataframe
        drop_col(lst): column names to drop from dataframe
    Return:
        df: a pandas dataframe without certain columns
    """
    LOGGER.info("Dropped columns: %s", drop_col)
    df.drop(columns=drop_col, inplace=True)
    return df

In [12]:
def drop_nans(df, drop_na_col):
    """Drop Nans from selected columns
    Args:
        df: a pandas dataframe
        drop_na_col(lst): column names to drop NaNs from 
    Return:
        df: a pandas dataframe without NaNs in certain columns
    """
    LOGGER.info("Dropped NaNs from these columns: %s", drop_na_col)
    df.dropna(subset=drop_na_col, inplace=True)
    return df

In [13]:
def filter_date_range(df, date_col="date"):
    """Filters and sorts the date column by specific date range in the dataframe.
    Args:
        df: a pandas dataframe
        date_col(string): column name with date information
    Returns:
        df: a pandas dataframe with a filtered date range
    """
    start_date = "2019-01-01"
    end_date = "2020-12-31"

    mask = (merge_all[date_col] > start_date) & (merge_all[date_col] <= end_date)
    df = df.loc[mask]
    df.sort_values([date_col], inplace=True)
    LOGGER.info("Filtered dataframe to only display 2019-2020 data")
    return df

In [14]:
def filter_df(df, fil_col, fil_val):
    """Creates a dataframe based on values from a single column
     Args:
        df: a pandas dataframe
        fil_col(string): column name from dataframe
        fil_val(string): values to restrict dataframe by
    Return:
        df: a pandas dataframe created by values from a single column
    """
    df = df[df[fil_col] == fil_val]
    return df

In [15]:
def create_container_df(df, container_col):
    """Creates container label based on original container column
    Args:
        df: a pandas dataframe
        container_col: a column name with the container label information
    Return:
        df: a pandas dataframe with a new column with container labels
    """
    df["container_label"] = df[container_col].str[0:2]
    LOGGER.info("Created a container_label column to show(ex.'PA')")
    return df

In [16]:
def create_cond_df(df, col, val):
    """Creates a dataframe based on values from a single column
     Args:
        df: a pandas dataframe
        col(string): column name from dataframe
        val(list): values to restrict dataframe by
    Return:
        df: a pandas dataframe created by values from a single column
    """
    df = df[df[col].str.contains("|".join(val))]
    LOGGER.info("Created a conditional dataframe based on a list of values")
    return df

##### Main

In [17]:
jem = read_file(path_jem, fields_jem)
ephys = read_file(path_ephys, fields_ephys)
shiny = read_file(path_shiny, fields_shiny)

INFO:__main__:Read file in as a pandas dataframe
INFO:__main__:Read file in as a pandas dataframe
INFO:__main__:Read file in as a pandas dataframe


In [19]:
jem = filter_df(jem, "status", "SUCCESS")
jem

,container,date,extraction.endPipetteR,extraction.postPatch,rigOperator,roi,status,organism_name
10,PAS4_171002_451_A01,2017-10-02 13:27:00 -0700,10.2,nucleus_absent,ramr,"VISp, layer 4",SUCCESS,Mouse
12,P8S4_171002_355_A01,2017-10-02 13:40:00 -0700,1400.0,nucleus_present,lindsayn,"VISp, layer 5",SUCCESS,Mouse
13,P8S4_171002_354_A01,2017-10-02 13:40:00 -0700,1100.0,nucleus_present,lindsayn,"VISp, layer 5",SUCCESS,Mouse
19,P9S4_171002_402_A01,2017-10-02 14:10:00 -0700,1000.0,nucleus_present,lisak,"VISp, layer 2/3",SUCCESS,Mouse
20,P2S4_171002_053_A01,2017-10-02 14:11:00 -0700,10.0,nucleus_absent,rustym,"VISp, layer 4",SUCCESS,Mouse
21,P2S4_171002_054_A01,2017-10-02 14:11:00 -0700,1000.0,nucleus_present,rustym,"VISp, layer 5",SUCCESS,Mouse
23,P2S4_171002_055_A01,2017-10-02 14:11:00 -0700,1000.0,nucleus_present,rustym,"VISp, layer 5",SUCCESS,Mouse
24,P2S4_171002_056_A01,2017-10-02 14:11:00 -0700,40.0,nucleus_absent,rustym,"VISp, layer 5",SUCCESS,Mouse
25,P8S4_171002_356_A01,2017-10-02 14:46:00 -0700,3000.0,nucleus_present,lindsayn,"VISp, layer 5",SUCCESS,Mouse
26,P8S4_171002_357_A01,2017-10-02 14:46:00 -0700,30.0,nucleus_absent,lindsayn,"VISp, layer 5",SUCCESS,Mouse


In [20]:
shiny

,sample_id,cell_name,cell_specimen_project,Tree_call,subclass_label,broad_class_label,topLeaf_label,marker_sum_norm_label,Norm_Marker_Sum.0.4_label,VISp_cluster
0,P6S4_170808_252_A01,Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-338917.03.01.02,T301x,Core,Meis2,GABAergic,2_Meis2,0.380674,False,Meis2 Adamts19
1,P2S4_171129_055_A01,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,mIVSCC-MET,Core,Meis2,GABAergic,2_Meis2,0.717450,True,Meis2 Adamts19
2,P1S4_170526_001_A01,Slc32a1-IRES-Cre;Ai14-321584.04.01.01,T301x,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.506820,True,n76
3,P1S4_170621_009_A01,Slc32a1-IRES-Cre;Ai14-326815.04.02.05,T301x,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.904325,True,Lamp5 Lhx6
4,P1S4_190923_004_A01,Sncg-IRES2-FlpO-neo;Ai65F-487894.09.01.01,mIVSCC-METx,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.853235,True,Lamp5 Lhx6
5,P2S4_180615_055_A01,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.906603,True,Lamp5 Lhx6
6,P2S4_180827_051_A01,Nkx2-1-CreERT2;Ai14-410065.03.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.934480,True,Lamp5 Lhx6
7,P2S4_181022_052_A01,Nos1-CreERT2;Ai14-421030.03.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.852529,True,Lamp5 Lhx6
8,P2S4_181022_053_A01,Nos1-CreERT2;Ai14-421030.02.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.901146,True,Lamp5 Lhx6
9,P8S4_171013_352_A01,Gad2-IRES-Cre;Ai14-352690.05.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.847397,True,Lamp5 Lhx6


In [21]:
merge_sj = merge_dataframes(shiny, jem, "sample_id", "container", "left")

INFO:__main__:Merged two pandas dataframe into one dataframe


In [22]:
merge_sj

,sample_id,cell_name,cell_specimen_project,Tree_call,subclass_label,broad_class_label,topLeaf_label,marker_sum_norm_label,Norm_Marker_Sum.0.4_label,VISp_cluster,container,date,extraction.endPipetteR,extraction.postPatch,rigOperator,roi,status,organism_name
0,P6S4_170808_252_A01,Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-338917.03.01.02,T301x,Core,Meis2,GABAergic,2_Meis2,0.380674,False,Meis2 Adamts19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P2S4_171129_055_A01,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,mIVSCC-MET,Core,Meis2,GABAergic,2_Meis2,0.717450,True,Meis2 Adamts19,P2S4_171129_055_A01,2017-11-29 14:46:00 -0700,10.0,nucleus_absent,rustym,VISp6a,SUCCESS,Mouse
2,P1S4_170526_001_A01,Slc32a1-IRES-Cre;Ai14-321584.04.01.01,T301x,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.506820,True,n76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P1S4_170621_009_A01,Slc32a1-IRES-Cre;Ai14-326815.04.02.05,T301x,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.904325,True,Lamp5 Lhx6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P1S4_190923_004_A01,Sncg-IRES2-FlpO-neo;Ai65F-487894.09.01.01,mIVSCC-METx,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.853235,True,Lamp5 Lhx6,P1S4_190923_004_A01,2019-09-23 13:25:38 -0700,13.0,nucleus_present,kristenh,HIPCA1,SUCCESS,Mouse
5,P2S4_180615_055_A01,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.906603,True,Lamp5 Lhx6,P2S4_180615_055_A01,2018-06-15 14:04:38 -0700,1000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse
6,P2S4_180827_051_A01,Nkx2-1-CreERT2;Ai14-410065.03.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.934480,True,Lamp5 Lhx6,P2S4_180827_051_A01,2018-08-27 12:17:26 -0700,1000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse
7,P2S4_181022_052_A01,Nos1-CreERT2;Ai14-421030.03.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.852529,True,Lamp5 Lhx6,P2S4_181022_052_A01,2018-10-22 14:35:30 -0700,2000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse
8,P2S4_181022_053_A01,Nos1-CreERT2;Ai14-421030.02.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.901146,True,Lamp5 Lhx6,P2S4_181022_053_A01,2018-10-22 16:00:36 -0700,2000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse
9,P8S4_171013_352_A01,Gad2-IRES-Cre;Ai14-352690.05.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.847397,True,Lamp5 Lhx6,P8S4_171013_352_A01,2017-10-13 11:11:00 -0700,1500.0,nucleus_present,lindsayn,"VISp, layer 2/3",SUCCESS,Mouse


In [23]:
merge_all = merge_dataframes(merge_sj, ephys, "cell_name", "name", "inner")

INFO:__main__:Merged two pandas dataframe into one dataframe


In [24]:
merge_all

,sample_id,cell_name,cell_specimen_project,Tree_call,subclass_label,broad_class_label,topLeaf_label,marker_sum_norm_label,Norm_Marker_Sum.0.4_label,VISp_cluster,container,date,extraction.endPipetteR,extraction.postPatch,rigOperator,roi,status,organism_name,name,vrest,ri,sag,tau,upstroke_downstroke_ratio_long_square,latency,f_i_curve_slope
0,P2S4_171129_055_A01,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,mIVSCC-MET,Core,Meis2,GABAergic,2_Meis2,0.717450,True,Meis2 Adamts19,P2S4_171129_055_A01,2017-11-29 14:46:00 -0700,10.0,nucleus_absent,rustym,VISp6a,SUCCESS,Mouse,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P2S4_180615_055_A01,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.906603,True,Lamp5 Lhx6,P2S4_180615_055_A01,2018-06-15 14:04:38 -0700,1000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,-73.571388,173.437506,0.070307,8.480884,4.265364,0.35832,0.207589
2,P2S4_180827_051_A01,Nkx2-1-CreERT2;Ai14-410065.03.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.934480,True,Lamp5 Lhx6,P2S4_180827_051_A01,2018-08-27 12:17:26 -0700,1000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse,Nkx2-1-CreERT2;Ai14-410065.03.01.01,-63.726659,202.812389,0.075065,12.192278,2.835392,0.34788,0.096618
3,P2S4_181022_052_A01,Nos1-CreERT2;Ai14-421030.03.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.852529,True,Lamp5 Lhx6,P2S4_181022_052_A01,2018-10-22 14:35:30 -0700,2000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse,Nos1-CreERT2;Ai14-421030.03.02.02,-65.250193,131.741062,0.016660,8.963292,4.029763,0.03658,0.276724
4,P2S4_181022_053_A01,Nos1-CreERT2;Ai14-421030.02.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.901146,True,Lamp5 Lhx6,P2S4_181022_053_A01,2018-10-22 16:00:36 -0700,2000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse,Nos1-CreERT2;Ai14-421030.02.01.01,-70.820336,264.218718,0.053235,10.512882,2.775232,0.02652,0.335000
5,P8S4_171013_352_A01,Gad2-IRES-Cre;Ai14-352690.05.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.847397,True,Lamp5 Lhx6,P8S4_171013_352_A01,2017-10-13 11:11:00 -0700,1500.0,nucleus_present,lindsayn,"VISp, layer 2/3",SUCCESS,Mouse,Gad2-IRES-Cre;Ai14-352690.05.02.01,-60.429114,273.549110,0.042446,11.895600,2.603708,0.00966,0.425000
6,P8S4_180808_352_A01,Gad2-IRES-Cre;Ai14-406388.04.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.971248,True,Lamp5 Lhx6,P8S4_180808_352_A01,2018-08-08 14:00:18 -0700,3000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Gad2-IRES-Cre;Ai14-406388.04.02.02,-59.537596,217.500001,0.034089,17.063826,2.544619,0.07506,0.212500
7,P8S4_180809_354_A01,Chrna2-Cre_OE25;Sst-IRES-FlpO;Ai65-406293.04.02.03,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.940367,True,Lamp5 Lhx6,P8S4_180809_354_A01,2018-08-09 12:38:01 -0700,2000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Chrna2-Cre_OE25;Sst-IRES-FlpO;Ai65-406293.04.02.03,-74.114178,157.812506,0.028230,10.232045,3.703602,0.12554,0.232530
8,P8S4_190517_351_A01,Nkx2-1-CreERT2;Ai14-465339.04.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.943339,True,Lamp5 Lhx6,P8S4_190517_351_A01,2019-05-17 10:44:56 -0700,2000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Nkx2-1-CreERT2;Ai14-465339.04.02.01,-76.165724,148.999944,0.034772,11.447166,3.936158,0.07302,0.225532
9,P9S4_180619_407_A01,Nos1-CreERT2;Ai14-396817.05.01.03,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.872356,True,Lamp5 Lhx6,P9S4_180619_407_A01,2018-06-19 13:18:15 -0700,1000.0,nucleus_present,lisak,VISp6a,SUCCESS,Mouse,Nos1-CreERT2;Ai14-396817.05.01.03,-71.218601,128.906250,0.031867,10.529962,3.458858,0.59866,0.135735


In [25]:
merge_all

,sample_id,cell_name,cell_specimen_project,Tree_call,subclass_label,broad_class_label,topLeaf_label,marker_sum_norm_label,Norm_Marker_Sum.0.4_label,VISp_cluster,container,date,extraction.endPipetteR,extraction.postPatch,rigOperator,roi,status,organism_name,name,vrest,ri,sag,tau,upstroke_downstroke_ratio_long_square,latency,f_i_curve_slope
0,P2S4_171129_055_A01,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,mIVSCC-MET,Core,Meis2,GABAergic,2_Meis2,0.717450,True,Meis2 Adamts19,P2S4_171129_055_A01,2017-11-29 14:46:00 -0700,10.0,nucleus_absent,rustym,VISp6a,SUCCESS,Mouse,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362357.04.01.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P2S4_180615_055_A01,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.906603,True,Lamp5 Lhx6,P2S4_180615_055_A01,2018-06-15 14:04:38 -0700,1000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse,Slc32a1-IRES-Cre;Ai14-393924.03.01.02,-73.571388,173.437506,0.070307,8.480884,4.265364,0.35832,0.207589
2,P2S4_180827_051_A01,Nkx2-1-CreERT2;Ai14-410065.03.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.934480,True,Lamp5 Lhx6,P2S4_180827_051_A01,2018-08-27 12:17:26 -0700,1000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse,Nkx2-1-CreERT2;Ai14-410065.03.01.01,-63.726659,202.812389,0.075065,12.192278,2.835392,0.34788,0.096618
3,P2S4_181022_052_A01,Nos1-CreERT2;Ai14-421030.03.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.852529,True,Lamp5 Lhx6,P2S4_181022_052_A01,2018-10-22 14:35:30 -0700,2000.0,nucleus_present,rustym,VISp5,SUCCESS,Mouse,Nos1-CreERT2;Ai14-421030.03.02.02,-65.250193,131.741062,0.016660,8.963292,4.029763,0.03658,0.276724
4,P2S4_181022_053_A01,Nos1-CreERT2;Ai14-421030.02.01.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.901146,True,Lamp5 Lhx6,P2S4_181022_053_A01,2018-10-22 16:00:36 -0700,2000.0,nucleus_present,rustym,VISp6b,SUCCESS,Mouse,Nos1-CreERT2;Ai14-421030.02.01.01,-70.820336,264.218718,0.053235,10.512882,2.775232,0.02652,0.335000
5,P8S4_171013_352_A01,Gad2-IRES-Cre;Ai14-352690.05.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.847397,True,Lamp5 Lhx6,P8S4_171013_352_A01,2017-10-13 11:11:00 -0700,1500.0,nucleus_present,lindsayn,"VISp, layer 2/3",SUCCESS,Mouse,Gad2-IRES-Cre;Ai14-352690.05.02.01,-60.429114,273.549110,0.042446,11.895600,2.603708,0.00966,0.425000
6,P8S4_180808_352_A01,Gad2-IRES-Cre;Ai14-406388.04.02.02,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.971248,True,Lamp5 Lhx6,P8S4_180808_352_A01,2018-08-08 14:00:18 -0700,3000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Gad2-IRES-Cre;Ai14-406388.04.02.02,-59.537596,217.500001,0.034089,17.063826,2.544619,0.07506,0.212500
7,P8S4_180809_354_A01,Chrna2-Cre_OE25;Sst-IRES-FlpO;Ai65-406293.04.02.03,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.940367,True,Lamp5 Lhx6,P8S4_180809_354_A01,2018-08-09 12:38:01 -0700,2000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Chrna2-Cre_OE25;Sst-IRES-FlpO;Ai65-406293.04.02.03,-74.114178,157.812506,0.028230,10.232045,3.703602,0.12554,0.232530
8,P8S4_190517_351_A01,Nkx2-1-CreERT2;Ai14-465339.04.02.01,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.943339,True,Lamp5 Lhx6,P8S4_190517_351_A01,2019-05-17 10:44:56 -0700,2000.0,nucleus_present,lindsayn,VISp5,SUCCESS,Mouse,Nkx2-1-CreERT2;Ai14-465339.04.02.01,-76.165724,148.999944,0.034772,11.447166,3.936158,0.07302,0.225532
9,P9S4_180619_407_A01,Nos1-CreERT2;Ai14-396817.05.01.03,mIVSCC-MET,Core,Lamp5,GABAergic,3_Lamp5 Lhx6,0.872356,True,Lamp5 Lhx6,P9S4_180619_407_A01,2018-06-19 13:18:15 -0700,1000.0,nucleus_present,lisak,VISp6a,SUCCESS,Mouse,Nos1-CreERT2;Ai14-396817.05.01.03,-71.218601,128.906250,0.031867,10.529962,3.458858,0.59866,0.135735


In [26]:
merge_all = filter_date_range(merge_all)

C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:__main__:Filtered dataframe to only display 2019-2020 data


In [27]:
merge_all

,sample_id,cell_name,cell_specimen_project,Tree_call,subclass_label,broad_class_label,topLeaf_label,marker_sum_norm_label,Norm_Marker_Sum.0.4_label,VISp_cluster,container,date,extraction.endPipetteR,extraction.postPatch,rigOperator,roi,status,organism_name,name,vrest,ri,sag,tau,upstroke_downstroke_ratio_long_square,latency,f_i_curve_slope
689,P1S4_190103_003_A01,Vip-IRES-Cre;Ai14-433569.03.01.02,mIVSCC-MET,Core,Vip,GABAergic,45_Vip,0.966677,True,Vip Pygm C1ql1,P1S4_190103_003_A01,2019-01-03 11:13:33 -0800,4000.0,nucleus_present,kristenh,VISp2/3,SUCCESS,Mouse,Vip-IRES-Cre;Ai14-433569.03.01.02,-71.890958,245.312572,0.030711,14.143285,3.756587,0.02856,0.148936
572,P1S4_190103_002_A01,Vip-IRES-Cre;Ai14-433569.03.01.01,mIVSCC-MET,Core,Vip,GABAergic,43_Vip,0.966119,True,Vip Crispld2 Kcne4,P1S4_190103_002_A01,2019-01-03 11:13:33 -0800,11400.0,nucleus_present,kristenh,VISp2/3,SUCCESS,Mouse,Vip-IRES-Cre;Ai14-433569.03.01.01,-54.746689,266.968727,0.233464,16.772145,2.077782,0.02914,0.420388
823,P8S4_190103_351_A01,Vip-IRES-Cre;Ai14-433569.04.02.01,mIVSCC-MET,Core,Vip,GABAergic,49_Vip,0.954358,True,Vip Lmo1 Myl1,P8S4_190103_351_A01,2019-01-03 11:27:09 -0800,3000.0,nucleus_present,lindsayn,VISp2/3,SUCCESS,Mouse,Vip-IRES-Cre;Ai14-433569.04.02.01,-65.221985,210.875049,0.057213,10.071189,3.611655,0.01336,0.057895
7279,P1S4_190104_001_A01,Sst-IRES-Cre;Ai14-434645.04.01.01,mIVSCC-MET,PoorQ,Sst,GABAergic,67_Sst,0.359904,False,n59,P1S4_190104_001_A01,2019-01-04 10:02:06 -0800,1500.0,nucleus_present,kristenh,VISp2/3,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.04.01.01,-72.817945,151.222393,0.158552,18.431600,1.897816,0.04548,0.315000
2059,P1S4_190104_002_A01,Sst-IRES-Cre;Ai14-434645.04.01.02,mIVSCC-MET,Core,Sst,GABAergic,95_Sst,0.991989,True,n91,P1S4_190104_002_A01,2019-01-04 10:02:06 -0800,2500.0,nucleus_present,kristenh,VISp5,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.04.01.02,-74.271588,223.593861,0.053998,17.967111,2.170470,0.04690,0.230000
2022,PFS4_190104_701_A01,Sst-IRES-Cre;Ai14-434645.03.01.01,mIVSCC-MET,I1,Sst,GABAergic,94_Sst,0.955347,True,Sst Calb2 Pdlim5,PFS4_190104_701_A01,2019-01-04 10:02:51 -0800,1500.0,nucleus_present,jessicat,VISp2/3,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.03.01.01,-66.458457,335.157156,0.200891,31.799607,1.956436,0.03484,0.410000
2155,PFS4_190104_702_A01,Sst-IRES-Cre;Ai14-434645.03.01.03,mIVSCC-MET,I1,Sst,GABAergic,95_Sst,1.006454,True,n91,PFS4_190104_702_A01,2019-01-04 10:02:51 -0800,1000.0,nucleus_present,jessicat,VISp5,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.03.01.03,-74.782421,115.125082,0.060572,11.667049,1.536190,0.04218,0.225000
6006,PBS4_190104_502_A01,Sst-IRES-Cre;Ai14-434645.05.01.02,mIVSCC-MET,PoorQ,Sst,GABAergic,68_Sst,0.979294,True,n91,PBS4_190104_502_A01,2019-01-04 10:15:51 -0800,1100.0,nucleus_present,dijonh,VISp4,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.05.01.02,-60.750129,207.499996,0.095045,21.534646,2.584531,0.03168,0.285000
6822,PBS4_190104_501_A01,Sst-IRES-Cre;Ai14-434645.05.01.01,mIVSCC-MET,PoorQ,L6 Car3,Glutamatergic,260_L6 Car3,0.534642,True,n15,PBS4_190104_501_A01,2019-01-04 10:15:51 -0800,2000.0,nucleus_present,dijonh,VISp2/3,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.05.01.01,-64.367810,283.593744,0.130827,24.642843,1.766052,0.04120,0.396552
1313,PAS4_190104_452_A01,Sst-IRES-Cre;Ai14-434645.03.02.02,mIVSCC-MET,Core,Sst,GABAergic,73_Sst,0.973815,True,Sst Rxfp1 Prdm8,PAS4_190104_452_A01,2019-01-04 10:27:27 -0800,1400.0,nucleus_present,ramr,VISp5,SUCCESS,Mouse,Sst-IRES-Cre;Ai14-434645.03.02.02,-72.584111,170.187488,0.054290,19.789761,2.390783,0.05026,0.275000


In [28]:
merge_all = create_container_df(merge_all, "container")

C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
INFO:__main__:Created a container_label column to show(ex.'PA')


In [29]:
merge_all.container_label.unique()

array(['P1', 'P8', 'PF', 'PB', 'PA', 'PE', 'P2'], dtype=object)

In [30]:
merge_all.sort_values(["date"], inplace=True)

C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
